<a href="https://colab.research.google.com/github/mohanrajmit/Transfer-Learning/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from pathlib import Path
import numpy as np
import joblib

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

In [1]:
!git clone https://github.com/mohanrajmit/Transfer-Learning.git

Cloning into 'Transfer-Learning'...
remote: Enumerating objects: 2226, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2226 (delta 0), reused 0 (delta 0), pack-reused 2223
Receiving objects: 100% (2226/2226), 13.86 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
!ls

sample_data  Transfer-Learning


In [4]:
# Load the training data set by looping over every image file
for image_file in Path("Transfer-Learning/training_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(224, 224))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [6]:
print(np.array(x_train).shape)

(2000, 224, 224, 3)


In [7]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = imagenet_utils.preprocess_input(x_train)



In [16]:

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))



In [17]:
feature_extractor.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [18]:
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)



63/63 [==============================] - 22s 162ms/step


In [19]:
features_x.shape

(2000, 7, 7, 512)

In [22]:
# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")

['y_train.dat']

In [23]:
# Load data set of extracted features
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")


In [21]:
Y_train = np.array(y_train)
print(x_train.shape[1:])
print(Y_train.shape)

(224, 224, 3)
(2000,)


In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [25]:
# Create a model and add layers
model = Sequential()

# Add layers to our model
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [26]:
# Train the model
model.fit(x_train,Y_train,epochs=10,shuffle=True, verbose=2)

# Save the trained model to a file so we can use it to make predictions later
model.save("bird_feature_classifier_model.h5")

Epoch 1/10
63/63 - 2s - loss: 1.7900 - accuracy: 0.6980 - 2s/epoch - 33ms/step
Epoch 2/10
63/63 - 0s - loss: 0.4109 - accuracy: 0.8170 - 242ms/epoch - 4ms/step
Epoch 3/10
63/63 - 0s - loss: 0.3134 - accuracy: 0.8585 - 216ms/epoch - 3ms/step
Epoch 4/10
63/63 - 0s - loss: 0.2563 - accuracy: 0.8875 - 227ms/epoch - 4ms/step
Epoch 5/10
63/63 - 0s - loss: 0.1944 - accuracy: 0.9190 - 229ms/epoch - 4ms/step
Epoch 6/10
63/63 - 0s - loss: 0.1820 - accuracy: 0.9265 - 222ms/epoch - 4ms/step
Epoch 7/10
63/63 - 0s - loss: 0.1543 - accuracy: 0.9300 - 224ms/epoch - 4ms/step
Epoch 8/10
63/63 - 0s - loss: 0.1602 - accuracy: 0.9305 - 233ms/epoch - 4ms/step
Epoch 9/10
63/63 - 0s - loss: 0.1324 - accuracy: 0.9460 - 218ms/epoch - 3ms/step
Epoch 10/10
63/63 - 0s - loss: 0.1088 - accuracy: 0.9540 - 229ms/epoch - 4ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [30]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("Transfer-Learning/test_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(224,224,3))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)

    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)

# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = imagenet_utils.preprocess_input(x_test)

In [31]:
x_test.shape

(200, 224, 224, 3)

In [32]:
# Load our trained classifier model
model = load_model("bird_feature_classifier_model.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model
predictions = model.predict(features_x)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

7/7 [==============================] - 0s 3ms/step
True Positives: 93
True Negatives: 79
False Positives: 21
False Negatives: 7
              precision    recall  f1-score   support

           0       0.92      0.79      0.85       100
           1       0.82      0.93      0.87       100

    accuracy                           0.86       200
   macro avg       0.87      0.86      0.86       200
weighted avg       0.87      0.86      0.86       200

